In [6]:
import pandas as pd
import numpy as np
import requests

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Tablas=pd.read_html(url)
toronto=Tablas[0]
toronto.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [3]:
toronto = toronto[~ (toronto.Borough == 'Not assigned')].reset_index(drop=True)

In [4]:
geocoor = pd.read_csv('Geospatial_Coordinates.csv')

In [5]:
toronto = toronto.join(geocoor.set_index('Postal Code'), on='PostalCode')

In [8]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [9]:
toronto_data = toronto.copy()
toronto_data['Neighborhood'] = toronto_data['Neighborhood'].apply(lambda x:x.split(',')[0])

In [23]:
toronto_data['Neighborhood'].nunique()

95

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                  )

In [16]:
toronto_venues['Venue Category'].nunique()

272

In [51]:
toronto_venues.Neighborhood.nunique()

93

In [17]:
toronto_venues['Venue Category'] = toronto_venues['Venue Category'].str.replace('Neighborhood', 'nbh')

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,nbh
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(2127, 273)

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,nbh
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
toronto_grouped.shape

(93, 273)

### create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,Alderwood,Pizza Place,Coffee Shop,Pub,Gym,Skating Rink,Sandwich Place,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant
2,Bathurst Manor,Bank,Coffee Shop,Shopping Mall,Sandwich Place,Sushi Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Restaurant,Mobile Phone Shop,Deli / Bodega
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,nbh,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,Bedford Park,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Pharmacy,Butcher,Café,Pub,Liquor Store


In [66]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 8, 0, 1, 1, 1, 1, 8, 4, 0, 1, 7, 1,
       1, 8, 1, 0, 1, 1, 5, 3, 1, 1, 1, 0, 1, 9, 1, 4, 1, 1, 0, 1, 1, 1,
       3, 1, 6, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 8, 8, 2, 1,
       1, 1, 1, 2, 0])

In [67]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

In [68]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Park,Bus Stop,nbh,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,8,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Theater,Café,Bank,Ice Cream Shop,Spa
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Coffee Shop,Concert Hall,Dim Sum Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Discount Store,Mexican Restaurant,Bank,Bar,Café,Diner,Beer Bar,Creperie


In [69]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Get the Coffee Shop info

In [81]:
venue_id = '5dba113267232c000803cb37' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [82]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '5dba113267232c000803cb37',
 'name': 'For Five Coffee Roasters',
 'contact': {},
 'location': {'lat': 40.71224,
  'lng': -74.014915,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.71224, 'lng': -74.014915}],
  'postalCode': '10280',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['New York, NY 10280', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/for-five-coffee-roasters/5dba113267232c000803cb37',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 7},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 10,
  'groups': [{'type': 'others', 'count': 10, 'items': []}],
  'summary': '10 Likes'},
 'dislike': False,
 'ok': Fal

In [83]:
print(result['response']['venue']['rating'])

7.6


In [86]:
coffee_lat = result['response']['venue']['location']['lat']
coffee_lng = result['response']['venue']['location']['lng']

In [96]:
all_data = toronto_data.append({'PostalCode':'',
                                'Borough':'', 
                                'Neighborhood':'For Five Coffee Roasters',
                                'Latitude':coffee_lat, 
                                'Longitude':coffee_lng}, ignore_index=True)

In [97]:
all_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South,43.636258,-79.498509
102,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999


In [98]:
all_venues = getNearbyVenues(names=all_data['Neighborhood'],
                                 latitudes=all_data['Latitude'],
                                 longitudes=all_data['Longitude']
                                  )

In [99]:
all_venues['Venue Category'] = all_venues['Venue Category'].str.replace('Neighborhood', 'nbh')

In [100]:
# one hot encoding
all_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
all_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_onehot.columns[-1]] + list(all_onehot.columns[:-1])
all_onehot = all_onehot[fixed_columns]

all_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,nbh
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
all_onehot.shape

(2209, 279)

In [102]:
all_grouped = all_onehot.groupby('Neighborhood').mean().reset_index()
all_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,nbh
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_grouped['Neighborhood']

for ind in np.arange(all_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Skating Rink,Lounge,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,Alderwood,Pizza Place,Gym,Coffee Shop,Dance Studio,Skating Rink,Sandwich Place,Pub,Dim Sum Restaurant,Deli / Bodega,Department Store
2,Bathurst Manor,Coffee Shop,Bank,Middle Eastern Restaurant,Fried Chicken Joint,Diner,Bridal Shop,Sandwich Place,Deli / Bodega,Restaurant,Supermarket
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,nbh,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,Bedford Park,Restaurant,Sushi Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Pharmacy,Butcher,Café,Pub


In [105]:
# set number of clusters
kclusters = 10

all_grouped_clustering = all_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 8, 0, 1, 1, 1, 1, 8, 4, 0, 1, 6,
       1, 1, 8, 1, 0, 1, 1, 5, 3, 1, 1, 1, 0, 1, 9, 1, 4, 1, 1, 0, 1, 1,
       1, 3, 1, 7, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 8, 8, 2,
       1, 1, 1, 1, 2, 0])

In [106]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
all_merged = all_data

In [107]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
all_merged = all_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
all_merged = all_merged[all_merged['Cluster Labels'].notna()]
all_merged['Cluster Labels'] = all_merged['Cluster Labels'].astype(int)
all_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Bus Stop,nbh,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,8,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Historic Site,Beer Store,Shoe Store
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Boutique,Diner,Department Store
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Burger Joint,Bank,Bar,Beer Bar,Italian Restaurant,Smoothie Shop,Sandwich Place,Distribution Center


### Clusters Visualization

In [180]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 1. Venues with the Coffee Shops as the 1st Most Common Venues

In [174]:
all_merged['Coffee Label'] = all_merged['1st Most Common Venue'].apply(lambda x: 1 if x== 'Coffee Shop' else 0)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Coffee Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 2. Venues with the Coffee Shops as the 2nd Most Common Venues

In [175]:
all_merged['Coffee Label'] = all_merged['2nd Most Common Venue'].apply(lambda x: 1 if x== 'Coffee Shop' else 0)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Coffee Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3. Venues with the Coffee Shops as the 3rd Most Common Venues

In [176]:
all_merged['Coffee Label'] = all_merged['3rd Most Common Venue'].apply(lambda x: 1 if x== 'Coffee Shop' else 0)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Coffee Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Get the Cluster Labels for the Coffee Shop

In [179]:
all_merged.loc[all_merged['Neighborhood'] == 'For Five Coffee Roasters', 'Cluster Labels']

103    1
Name: Cluster Labels, dtype: int32

### Show the Nighborhood on the same cluster

In [170]:
all_merged.loc[all_merged['Cluster Labels'] == 1, all_merged.columns[[2] + list(range(6, all_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Coffee Label
2,Regent Park,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Historic Site,Beer Store,Shoe Store,0
3,Lawrence Manor,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Boutique,Diner,Department Store,0
4,Queen's Park,Coffee Shop,Sushi Restaurant,Burger Joint,Bank,Bar,Beer Bar,Italian Restaurant,Smoothie Shop,Sandwich Place,Distribution Center,0
7,Don Mills,Coffee Shop,Asian Restaurant,Gym,Beer Store,Japanese Restaurant,Restaurant,Sporting Goods Shop,Baseball Field,Dim Sum Restaurant,Discount Store,0
8,Parkview Hill,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pharmacy,Breakfast Spot,Bank,Pet Store,0
...,...,...,...,...,...,...,...,...,...,...,...,...
97,First Canadian Place,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Deli / Bodega,Salad Place,Steakhouse,Asian Restaurant,0
99,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Burger Joint,Pub,Men's Store,Bubble Tea Shop,0
100,Business reply mail Processing Centre,Light Rail Station,Gym / Fitness Center,Garden,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio,0
102,Mimico NW,Grocery Store,Fast Food Restaurant,Supplement Shop,Bakery,Tanning Salon,Burger Joint,Flower Shop,Sandwich Place,Discount Store,Convenience Store,0


In [197]:
first_cluster = all_merged.loc[all_merged['Cluster Labels'] == 1].copy()
first_cluster['Coffee Label'] = 0

In [203]:
def labelCoffee(x):
    for col in x[6:]: 
        if col == 'Coffee Shop' or col == 'Café':
            return 1
    return 0    
first_cluster['Coffee Label'] = first_cluster.apply(labelCoffee, axis=1)

In [204]:
# first_cluster[first_cluster['Coffee Label'] == 1]
first_cluster['Coffee Label'].value_counts()

1    52
0    24
Name: Coffee Label, dtype: int64

In [205]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(first_cluster['Latitude'], first_cluster['Longitude'], first_cluster['Neighborhood'], first_cluster['Coffee Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Find the location with least coffee shop

In [142]:
fit_location = all_merged.loc[all_merged['Cluster Labels'] == 1, all_merged.columns[[2] + list(range(6, all_merged.shape[1]))]].reset_index(drop=True)

In [143]:
best_location = fit_location

In [144]:
for col in best_location.columns[1:]:
    best_location = best_location[~(best_location[col].str.contains('Coffee')) & ~(best_location[col].str.contains('Café'))]

In [145]:
len(best_location)

24

In [147]:
best_location.reset_index(drop=True, inplace=True)

In [150]:
best_location.Neighborhood.tolist()

['Parkview Hill',
 'Woodbine Heights',
 'Guildwood',
 'Cedarbrae',
 'Hillcrest Village',
 'Thorncliffe Park',
 'Downsview',
 'Downsview',
 'India Bazaar',
 'Cliffside',
 'Downsview',
 'Del Ray',
 'Downsview',
 'Roselawn',
 'Dorset Park',
 'Davisville North',
 'Forest Hill North & West',
 'Wexford',
 'Agincourt',
 'Clarks Corners',
 'Moore Park',
 'South Steeles',
 'Business reply mail Processing Centre',
 'Mimico NW']